In [48]:
!pip install pandas transformers torch underthesea vncorenlp


[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [49]:
from underthesea import word_tokenize, pos_tag
import numpy as np
import math
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
model = AutoModelForMaskedLM.from_pretrained("vinai/phobert-base")

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


#### Perplexity là gì?

**Perplexity (PPL)** là một chỉ số đo độ “bối rối” của mô hình ngôn ngữ khi dự đoán từ tiếp theo trong câu.  

Nói cách khác: **mức độ khó dự đoán một chuỗi văn bản**.  
Công thức cơ bản cho một câu w_1, w_2, ..., w_N  với mô hình LM:

![alt text](image.png)

- *PPL >=1* 
- *PPL = 1* → câu quá dễ, mô hình dự đoán hoàn hảo (hiếm xảy ra).
- *PPL cao* → câu khó dự đoán, nhiều khả năng từ vựng khác nhau → độ phức tạp cao..


In [50]:
def compute_perplexity(sentence):
    encodings = tokenizer(sentence, return_tensors="pt", truncation=True, max_length=256)
    input_ids = encodings.input_ids
    target_ids = input_ids.clone()
    with torch.no_grad():
        outputs = model(input_ids, labels=target_ids)
        loss = outputs.loss
    return math.exp(loss.item())

### Feature đánh giá độ phức tạp câu

##### Feature thống kê cú pháp (syntactic/structural features)

| Feature | Kiểu | Mô tả | Ví dụ |
|---------|------|-------|-------|
| sentence_length | Numeric | Số từ trong câu | 13 |
| avg_word_length | Numeric | Độ dài trung bình của từ | 3.3 |
| num_clauses | Numeric | Số mệnh đề ước lượng bằng liên từ | 0 |
| num_punct | Numeric | Số dấu câu | 0 |
| num_nouns | Numeric | Số danh từ (`N*`) | 8 |
| num_verbs | Numeric | Số động từ (`V*`) | 2 |
| num_adjs | Numeric | Số tính từ (`A*`) | 0 |


In [51]:
def extract_features(sentence):
    tokens = word_tokenize(sentence)
    sentence_length = len(tokens)
    avg_word_length = np.mean([len(t) for t in tokens]) if tokens else 0

    pos = pos_tag(sentence)
    num_nouns = sum(1 for _, p in pos if p.startswith("N"))
    num_verbs = sum(1 for _, p in pos if p.startswith("V"))
    num_adjs = sum(1 for _, p in pos if p.startswith("A"))

    connectors = {"và","hoặc", "nhưng","tuy nhiên","cũng như là","nên", "mà", 
                  "nếu", "thì", "bởi vì", "vì" ,"khi", 
                  "mặc dù", "để","sau","sau_khi", 
                  "trước", "trước khi", "hay", "khi","do", "do đó",
                  "hễ", "lẫn", "cùng", "ngoài ra", "vậy", "trừ phi",
                  "hơn", "bằng", "như", "dù cho", "nhằm", "vì vậy",
                  "ngược lại", "chỉ" ,"chỉ trừ"}
    num_clauses = sum(1 for t in tokens if t.lower() in connectors)
    num_punct = sum(1 for t in tokens if t in ".,;!?")

    # Bỏ tree_depth, num_long_dep vì underthesea không parse_dep
    perplexity = compute_perplexity(sentence)

    return [
        sentence_length, avg_word_length, num_clauses, num_punct,
        num_nouns, num_verbs, num_adjs, perplexity
    ]

In [52]:
import json
import pandas as pd

input_file = r"D:\NCKH\estimate_question_difficulty\Embedding\Output_ws\ori_questions.json"
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

rows = []
for item in data:
    qid = item["id"]
    subject = item["subject"]
    question = item["question"]
    
    feat = extract_features(question)
    rows.append([qid, subject, question] + feat)

columns = ["id", "subject", "question",
           "sentence_length", "avg_word_length", "num_clauses", "num_punct",
           "num_nouns", "num_verbs", "num_adjs", "perplexity"]

df = pd.DataFrame(rows, columns=columns)

output_file = r"D:\NCKH\estimate_question_difficulty\Embedding\Output_features\question_features.csv"
df.to_csv(output_file, index=False, encoding="utf-8-sig")
print("✅ Đã xuất CSV:", output_file)
print(df.head())

✅ Đã xuất CSV: D:\NCKH\estimate_question_difficulty\Embedding\Output_features\question_features.csv
   id  subject                                           question  \
0   1  Văn học  Từ Văn hóa trong văn bản Phong cách Hồ Chí Min...   
1   2  Văn học  Ý nào nói lên việc tiếp thu có chọn lọc tinh h...   
2   3  Văn học  Vấn đề chủ yếu được nói tới trong văn bản Phon...   
3   4  Văn học  Theo tác giả quan niệm thẩm mĩ về cuộc sống củ...   
4   5  Văn học         Từ nào sau đây trái nghĩa với truân chuyên   

   sentence_length  avg_word_length  num_clauses  num_punct  num_nouns  \
0                8         6.250000            0          0          4   
1               19         5.000000            0          0          7   
2               11         5.454545            0          0          4   
3               11         5.818182            0          0          5   
4                8         4.375000            1          0          2   

   num_verbs  num_adjs  perplexity  
0  